In [4]:
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import pandas as pd
from ta.volatility import BollingerBands
from ta.momentum import StochRSIIndicator
from ta.trend import MACD
from tqdm import tqdm
from gensim.models import Word2Vec
from itertools import combinations
from prophet import Prophet

In [35]:
datetime.today().strftime('%Y-%m-%d')

'2024-01-11'

In [28]:
def get_us_code(x):
    # x:0 미국이 주말이 아닐 때, x:1 전날이 미국이 주말일 때
    if x == 0:
        # 오늘 날짜
        oneday = datetime.today().strftime('%Y-%m-%d')
        # 이틀 전 날짜
        twoday = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')
    elif x == 1:
        # 오늘 날짜
        oneday = (datetime.today() - timedelta(days=4)).strftime('%Y-%m-%d')
        # 이틀 전 날짜
        twoday = (datetime.today() - timedelta(days=6)).strftime('%Y-%m-%d')
    
    oneday = datetime.today().strftime('%Y-%m-%d')
    # 이틀 전 날짜
    twoday = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')
    
    df_NASDAQ = fdr.StockListing('NASDAQ')[:40]
    
    # df_US_IT = df_NASDAQ[df_NASDAQ['Industry'] == '소프트웨어'].reset_index(drop=True)
    
    codes = []
    top_percentile = 5  # 양수의 변동률 중에서 상위 몇 퍼센트를 가져올지 설정
    
    for idx, row in df_NASDAQ.iterrows():
        code = row['Symbol']
        name = row['Name']
        # 한국 시장이 열리기 바로 전 미국시장 주가 데이터 수집
        try:
            price = fdr.DataReader(code, start=twoday, end=oneday)['Close']
            pct_change = (price.pct_change() * 100)[-1]  # pct_change 계산
            print(f"{code}, {name}: {pct_change:.2f}%")
            codes.append((code, pct_change))    

        except:
            pass
    
    codes.sort(key=lambda x: x[1], reverse=True)
    num_codes = int(len(codes) * top_percentile / 100)
    
    top_codes = codes[:num_codes]
    top_codes = [code for code, _ in top_codes]

    return top_codes, oneday, twoday

def make_us_kr_set(codes):
    print("한국종목과 데이터 셋을 만들고 있습니다.")

    # 가져오는 데이터는 18년1월1일로 설정
    start_date = '2018-01-01'
    end_date = today
    df_ksq = fdr.StockListing('KOSDAQ')[:40]

    # 미국 종목 데이터 가져오기
    codes_data = [fdr.DataReader(code, start_date, end_date)['Close'].rename(str(code + '_U'))
                  for code in codes]
    # 한국 종목 데이터 가져오기
    ksq_data = [fdr.DataReader(code, start_date, end_date)['Close'].rename(str(code + '_K'))
                for code in df_ksq['Code']]

    df = pd.concat(codes_data + ksq_data, axis=1, sort=True)
    df.index.name = 'Date'
    # 결측치를 중간값으로 채우기
    df.fillna(df.median(), inplace=True)
    return df

def get_high_correlations(x, threshold_low=0, threshold_high=1):
    # 최소 상관계수 0, 최대 상관계수 1로 설정
    print('피어슨 상관관계 분석: 상위 20개 종목 ')
    # 상관관계 계산
    corr_matrix = x.corr()
    # 상관계수 내림차순 정렬
    high_corr_pairs = corr_matrix.unstack().sort_values(ascending=False)
    # 중복 주식 쌍 필터
    high_corr_pairs = high_corr_pairs[
        high_corr_pairs.index.get_level_values(0) < high_corr_pairs.index.get_level_values(1)]
    # 상관계수 범위 값 내 쌍들만 저장
    high_corr_pairs = high_corr_pairs[
        (high_corr_pairs >= threshold_low) & (high_corr_pairs <= threshold_high)].reset_index()
    # 컬러명 설정
    high_corr_pairs.columns = ['Company 1', 'Company 2', 'Correlation']

    # 상관계수 상위 20개 종목 추출
    high_corr_pairs = high_corr_pairs[
                          high_corr_pairs['Company 1'].str.contains('_K') & high_corr_pairs['Company 2'].str.contains(
                              '_U')][:20]
    # 한국주식만 추출
    high_corr_pairs['Company 1'] = high_corr_pairs['Company 1'].str.extract('(\d+)')
    high_corr_pairs = high_corr_pairs.reset_index(drop=True)

    return high_corr_pairs['Company 1'].tolist()


def prophetmodel(x):
    yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

    kr_list=[] # 빈 리스트

    for kr in x:
        # 모델생성
        prophet=Prophet(
            growth='logistic',
            changepoint_range=0.7,# 변동이 발생하는 비율 (높을수록 변동 발생 비율을 높다고 판단->과적합 우려)
            changepoint_prior_scale=0.7, # 변동 발생 민감도(높을수록 변동 발생을 민감하게 잡아냄 ->역시 과적합 우려)
            interval_width=0.95, # 신뢰구간
            daily_seasonality=8,
            yearly_seasonality=12)

        # 데이터 받아오기
        df = fdr.DataReader(symbol=kr, start='2018',end=yesterday)

        # 데이터프레임 내의 날짜, 종가 열명을 변경
        prophet_series=pd.DataFrame(list(zip(df.index,df['Close'])),columns=['ds','y'])

        # 모델의 상한, 하한 설정
        prophet_series['cap']=prophet_series['y'].max() # 종가 데이터의 최대값을 상한선으로 지정
        prophet_series['floor']=prophet_series['y'].min() # 종가 데이터의 최소값을 하한선으로 지정

        # 학습
        prophet.fit(prophet_series)

        # 받아온 데이터로 예측
        forecast=prophet.predict(prophet_series)

        # 미래 예측(날짜 데이터 생성)
        future=prophet.make_future_dataframe(periods=1, freq='d')  # 1일 후의 종가 데이터 예측

        future['cap']=prophet_series['y'].max() *1.1
        future['floor']=prophet_series['y'].min() *0.9

        # 종가가 상승한 주식은 리스트에 추가
        if forecast['trend'].iloc[-1]>forecast['trend'].iloc[-2]: # 예측 전날 데이터가 예측 당일 데이터보다 종가가 큰 경우 리스트에 추가
            kr_list.append(kr)

    return kr_list

In [29]:
def signal(kospi_list):
    buys = {}	    # 매수 종목 저장
    returns = {}	# 각 종목 수익률 저장

    for ticker in kospi_list:		# KOSPI 종목 수집
        if len(ticker) != 6:
            continue
        df = fdr.DataReader(ticker, '2018-01-01', today)

        # 종가 기준 Bollinger Band 지표 계산(20일 이동평균, 20일 동안의 주가 표준편차 값*2)
        bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
        df['bb_high'] = bb.bollinger_hband()
        df['bb_low'] = bb.bollinger_lband()
        middle_bb = (df['High'] + df['Low'] + df['Close']) / 3
        middle_bb = middle_bb.rolling(window=20).mean()
        bb_width = (df['bb_high'] - df['bb_low']) / middle_bb * 100

        # 종가 기준 스토캐스틱 RSI 지표 계산(14일 이동평균, smooth값은 지표의 부드러운 정도)
        srsi = StochRSIIndicator(close=df['Close'], window=14, smooth1=3, smooth2=3)
        df['%K'] = srsi.stochrsi()

        # 종가 기준 MACD 지표 계산(장기이동평균=26일, 단기이동평균=12일, MACD 9일 단순이동평균)
        macd = MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9)
        macd_values = macd.macd()
        macd_signal = macd.macd_signal()
        macd_diff = macd_values - macd_signal

        # SRSI 20일로 설정
        n_days = 20

        # Bollinget Band, SRSI(%K) 지표 설정 수치에 맞춰 매수 신호 알림
        if bb_width.iloc[-1] >= 5 and df['%K'].iloc[-1] <= 0.25:   # 2달 전 한 달 동안의 RSI 지표 추출
            prev_month_rsi = df['%K'].iloc[-n_days:-n_days // 2]
            prev_month_rsi_mean = prev_month_rsi.mean()
            if prev_month_rsi_mean <= 0.25 and df['%K'].iloc[-n_days // 2] >= 0.25:   # 한 달 전의 RSI 지표 확인
                if macd_diff.iloc[-1] > macd_signal.iloc[-1]:
                    buys[ticker] = df['Close'].iloc[-1]
                    returns[ticker] = 0.0
                    print(f"{ticker}: 매수")
                else:
                    print(f"{ticker}: RSI 충족 BUT MACD 미충족")
            else:
                print(f"{ticker}: RSI 미충족")
        else:
            print(f"{ticker}: 매수 위험")

In [19]:
codes, today = get_us_code(0)

100%|██████████| 3879/3879 [00:09<00:00, 412.97it/s]


AAPL, Apple Inc: 0.57%
MSFT, Microsoft Corp: 1.86%
AMZN, Amazon.com Inc: 1.56%
NVDA, NVIDIA Corp: 2.28%
GOOGL, Alphabet Inc Class A: 0.94%
META, Meta Platforms Inc: 3.65%
GOOG, Alphabet Inc Class C: 0.87%
TSLA, Tesla Inc: -0.43%
AVGO, Broadcom Inc: -0.18%
COST, Costco Wholesale Corp: 0.84%
ASML, ASML Holding NV ADR: 0.26%
ADBE, Adobe Inc: 0.82%
AMD, Advanced Micro Devices Inc: -0.48%
PEP, PepsiCo Inc: -0.15%
AZN, AstraZeneca PLC ADR: 0.96%
NFLX, Netflix Inc: -0.78%
CSCO, Cisco Systems Inc: 0.66%
INTC, Intel Corp: -1.21%
PDD, PDD Holdings Inc ADR: -0.86%
LIN, Linde PLC: -0.20%
TMUS, T-Mobile US Inc: 0.06%
CMCSA, Comcast Corp: 0.46%
INTU, Intuit Inc: 0.58%
AMGN, Amgen Inc: -0.88%
QCOM, Qualcomm Inc: -0.41%
TXN, Texas Instruments Inc: -0.82%
HON, Honeywell International Inc: 0.38%
SNY, Sanofi SA ADR: 1.26%
AMAT, Applied Materials Inc: -0.81%
BKNG, Booking Holdings Inc: 0.91%
ISRG, Intuitive Surgical Inc: 10.25%
VRTX, Vertex Pharmaceuticals Inc: 0.65%
GILD, Gilead Sciences Inc: -1.81%
SBUX

In [20]:
df = make_us_kr_set(codes)

한국종목과 데이터 셋을 만들고 있습니다.


In [30]:
corr = get_high_correlations(df)

피어슨 상관관계 분석: 상위 20개 종목 


In [32]:
df_prophet = prophetmodel(corr)

05:33:55 - cmdstanpy - INFO - Chain [1] start processing
05:33:57 - cmdstanpy - INFO - Chain [1] done processing
05:33:58 - cmdstanpy - INFO - Chain [1] start processing
05:33:59 - cmdstanpy - INFO - Chain [1] done processing
05:33:59 - cmdstanpy - INFO - Chain [1] start processing
05:34:01 - cmdstanpy - INFO - Chain [1] done processing
05:34:02 - cmdstanpy - INFO - Chain [1] start processing
05:34:04 - cmdstanpy - INFO - Chain [1] done processing
05:34:04 - cmdstanpy - INFO - Chain [1] start processing
05:34:06 - cmdstanpy - INFO - Chain [1] done processing
05:34:07 - cmdstanpy - INFO - Chain [1] start processing
05:34:09 - cmdstanpy - INFO - Chain [1] done processing
05:34:10 - cmdstanpy - INFO - Chain [1] start processing
05:34:11 - cmdstanpy - INFO - Chain [1] done processing
05:34:12 - cmdstanpy - INFO - Chain [1] start processing
05:34:14 - cmdstanpy - INFO - Chain [1] done processing
05:34:15 - cmdstanpy - INFO - Chain [1] start processing
05:34:18 - cmdstanpy - INFO - Chain [1]

In [33]:
# 피어슨 상관관계 추천 알고리즘
model_1 = signal(df_prophet)

067310: RSI 미충족
035900: RSI 미충족
095340: 매수 위험
121600: 매수 위험
214150: RSI 미충족
078600: RSI 미충족
058470: RSI 충족 BUT MACD 미충족
036930: 매수 위험
247540: 매수 위험
041510: 매수 위험
078600: RSI 미충족
067310: RSI 미충족
065350: 매수 위험
058470: RSI 충족 BUT MACD 미충족
214150: RSI 미충족
086520: 매수 위험
214370: RSI 미충족


In [87]:
# df_pct = round(df.pct_change().shift(-1)  * 100, 4)
# df_pct.fillna(df_pct.median(), inplace=True)

In [126]:
airs['Company 1'].tolist()

In [167]:

print(df_prophet)

05:16:09 - cmdstanpy - INFO - Chain [1] start processing
05:16:11 - cmdstanpy - INFO - Chain [1] done processing
05:16:12 - cmdstanpy - INFO - Chain [1] start processing
05:16:13 - cmdstanpy - INFO - Chain [1] done processing
05:16:14 - cmdstanpy - INFO - Chain [1] start processing
05:16:16 - cmdstanpy - INFO - Chain [1] done processing
05:16:16 - cmdstanpy - INFO - Chain [1] start processing
05:16:18 - cmdstanpy - INFO - Chain [1] done processing
05:16:19 - cmdstanpy - INFO - Chain [1] start processing
05:16:20 - cmdstanpy - INFO - Chain [1] done processing
05:16:21 - cmdstanpy - INFO - Chain [1] start processing
05:16:24 - cmdstanpy - INFO - Chain [1] done processing
05:16:25 - cmdstanpy - INFO - Chain [1] start processing
05:16:26 - cmdstanpy - INFO - Chain [1] done processing
05:16:26 - cmdstanpy - INFO - Chain [1] start processing
05:16:27 - cmdstanpy - INFO - Chain [1] done processing
05:16:28 - cmdstanpy - INFO - Chain [1] start processing
05:16:30 - cmdstanpy - INFO - Chain [1]

['000270', '207940', '000660', '005490', '005930', '005380', '005935', '051910', '005935', '000660', '207940', '005930', '000270', '005380', '051910', '373220']


In [170]:
# 피어슨 상관관계 추천 알고리즘
model_1 = signal(df_prophet)

000270: RSI 미충족
207940: 매수 위험
000660: 매수 위험
005490: RSI 미충족
005930: RSI 미충족
005380: RSI 미충족
005935: RSI 미충족
051910: RSI 미충족
005935: RSI 미충족
000660: 매수 위험
207940: 매수 위험
005930: RSI 미충족
000270: RSI 미충족
005380: RSI 미충족
051910: RSI 미충족
373220: 매수 위험


In [148]:
prophet_series = pd.DataFrame(list(zip(df.index,df['Close'])), columns=['ds', 'y'])

In [152]:
# 모델의 상한, 하한 설정
prophet_series['cap']=prophet_series['y'].max() # 종가 데이터의 최대값을 상한선으로 지정
prophet_series['floor']=prophet_series['y'].min() # 종가 데이터의 최소값을 하한선으로 지정

In [154]:
prophet.fit(prophet_series)

05:10:28 - cmdstanpy - INFO - Chain [1] start processing
05:10:30 - cmdstanpy - INFO - Chain [1] done processing


In [155]:
# 받아온 데이터로 예측
forecast=prophet.predict(prophet_series)

In [158]:
forecast

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-01-02,53224.733453,101500,21500,20582.480321,40267.116761,53224.733453,53224.733453,-23099.049441,-23099.049441,...,-449.345601,-449.345601,-449.345601,-2218.279840,-2218.279840,-2218.279840,0.0,0.0,0.0,30125.684012
1,2018-01-03,53226.099652,101500,21500,20666.371843,39535.467103,53226.099652,53226.099652,-23092.435796,-23092.435796,...,-386.686552,-386.686552,-386.686552,-2274.325244,-2274.325244,-2274.325244,0.0,0.0,0.0,30133.663856
2,2018-01-04,53227.465871,101500,21500,19911.237563,40716.276698,53227.465871,53227.465871,-23232.657636,-23232.657636,...,-482.132800,-482.132800,-482.132800,-2319.100836,-2319.100836,-2319.100836,0.0,0.0,0.0,29994.808234
3,2018-01-05,53228.832110,101500,21500,19684.761649,39582.238943,53228.832110,53228.832110,-23293.464448,-23293.464448,...,-511.634103,-511.634103,-511.634103,-2350.406345,-2350.406345,-2350.406345,0.0,0.0,0.0,29935.367661
4,2018-01-08,53232.930947,101500,21500,19638.975454,40208.572524,53232.930947,53232.930947,-23499.436451,-23499.436451,...,-724.131317,-724.131317,-724.131317,-2343.881134,-2343.881134,-2343.881134,0.0,0.0,0.0,29733.494496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2024-01-04,101499.999999,101500,21500,68195.662296,88513.437219,101499.999999,101499.999999,-23211.818947,-23211.818947,...,-482.132800,-482.132800,-482.132800,-2298.262147,-2298.262147,-2298.262147,0.0,0.0,0.0,78288.181052
1480,2024-01-05,101499.999999,101500,21500,68517.094165,87175.138864,101499.999999,101499.999999,-23279.627885,-23279.627885,...,-511.634103,-511.634103,-511.634103,-2336.569782,-2336.569782,-2336.569782,0.0,0.0,0.0,78220.372114
1481,2024-01-08,101499.999999,101500,21500,68099.130516,87194.623897,101499.999999,101499.999999,-23512.249948,-23512.249948,...,-724.131317,-724.131317,-724.131317,-2356.694631,-2356.694631,-2356.694631,0.0,0.0,0.0,77987.750051
1482,2024-01-09,101499.999999,101500,21500,67923.353146,88462.316359,101499.999999,101499.999999,-23206.723592,-23206.723592,...,-449.345601,-449.345601,-449.345601,-2325.953991,-2325.953991,-2325.953991,0.0,0.0,0.0,78293.276408
